# **1. 라이브러리 및 기본 설정**

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

# **2. 임베딩**
- 데이터 임베딩에 대한 간단한 예제

In [2]:
## 단어를 인덱스에 매핑하는 딕셔너리
word_to_ix = {"hello": 0, "world": 1}

## 임베딩 레이어 생성(2개의 단어, 5차원 임베딩)
# nn.Embedding 사용
embeds = nn.Embedding(2, 5)

In [3]:
## 임베딩 벡터 생성
# 특정 단어에 대한 텐서를 생성하고 임베딩 레이어를 통해 벡터를 조회
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed) # "hello" 단어의 임베딩 벡터 출력

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward0>)


# **3. 데이터 준비**

In [4]:
# 컨텍스트 크기/ 임베딩 차원 정의
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

# Shakespeare 텍스트를 단어 단위로 분리하여 N-Gram 데이터로 저장
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [5]:
## N-Gram 데이터 생성
# 원래는 입력을 토큰화해야하지만 해당 예제에서는 튜플 리스트를 생성
# 각 튜플은 ([word_i-CONTEXT_SIZE, ..., word_i-1], 대상 단어) 형태

ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)], # 컨텍스트 단어들
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]

# 처음 3개의 데이터 확인하기
print(ngrams[:3])

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]


In [6]:
# 전체 텍스트에서 고유 단어를 추출하여 vocabulary map 생성
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

# **4. 모델링**
- n-gram 언어 모델 정의

In [7]:
## N-Gram 언어 모델 클래스 정의
# 임베딩 레이어와 두 개의 선형 계층(Linear)으로 구성된 N-Gram 언어 모델
class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        ## 임베딩 레이어 생성
        # 주어진 "어휘 크기"와 "임베딩 차원"을 기반으로 랜덤하게 초기화된 임베딩 행렬 생성
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)

        ## 첫 번째 선형 계층
        # input_dim: context_size * embedding_dim, output_dim: 128
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)

        ## 두 번째 선형 계층
        # 첫 번째 선형 계층의 출력을 받아 어휘집 크기로 출력
        self.linear2 = nn.Linear(128, vocab_size)


    def forward(self, inputs):
        # 임베딩 레이어를 통과하여 입력 벡터를 펼침(flatten)
        embeds = self.embeddings(inputs).view((1, -1))
        # 첫 번째 선형 계층 통과 및 활성화 함수 적용
        out = F.relu(self.linear1(embeds))
        # 두 번째 선형 계층 통과(활성화 함수 적용 x)
        out = self.linear2(out)
        # 출력층: 로그 소프트맥스를 통해 proportion 계산
        log_probs = F.log_softmax(out, dim=1)

        return log_probs

# **5. 학습**

In [8]:
## 손실 함수 및 최적화 함수(optimizer) 설정
losses = []
loss_function = nn.NLLLoss() # negative log-likelihood
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE) # 모델 객체 생성
optimizer = optim.SGD(model.parameters(), lr=0.001) # 경사 하강법 활용

In [9]:
for epoch in range(10):
    # 각 epoch에서의 total_loss 초기화
    total_loss = 0

    # 각 N-Gram 데이터를 활용하여..
    for context, target in ngrams:
        # vocabulary mapping(컨텍스트 단어 → 정수 인덱스)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        ## forward pass
        # 그래디언트 초기화
        model.zero_grad()
        # log probability 계산
        log_probs = model(context_idxs)
        # 손실 계산(대상 단어의 인덱스를 사용)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        ## backward pass
        loss.backward()
        optimizer.step()
        total_loss += loss.item() # 현재 손실을 총 손실에 추가
    # 각 에포크의 손실 저장
    losses.append(total_loss)
print(losses)  # 에포크 별 손실 출력 (훈련이 진행됨에 따라 감소)

[519.2403349876404, 516.7696256637573, 514.3148913383484, 511.8742139339447, 509.44712257385254, 507.03318881988525, 504.6318883895874, 502.24181365966797, 499.86163806915283, 497.4908583164215]


In [10]:
## 학습된 임베딩 확인
# 특정 단어의 임베딩 결과 확인(예: "beauty")
print(model.embeddings.weight[word_to_ix["beauty"]])

tensor([ 0.3439, -0.5519, -0.9614,  0.2754, -0.6782, -0.5463,  1.1518, -0.3991,
         0.9745,  1.5451], grad_fn=<SelectBackward0>)
